In [1]:
!pip install kaggle datasets transformers torch torchvision

In [2]:
# Import necessary libraries
import os
import zipfile
import shutil
from PIL import Image
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, random_split
from torch.optim import AdamW
from sklearn.metrics import accuracy_score, f1_score
from tqdm import tqdm
from transformers import CLIPProcessor, CLIPModel

In [3]:
# Early preprocessing of the data which includes signs other than the alphabet and a test dataset which is not usable due to its small size.

# Download the dataset from Kaggle
!kaggle datasets download -d debashishsau/aslamerican-sign-language-aplhabet-dataset

# Unzip the file
with zipfile.ZipFile('/content/aslamerican-sign-language-aplhabet-dataset.zip', 'r') as zip_ref:
    zip_ref.extractall('/content/aslamerican-sign-language-aplhabet-dataset')

# Remove images which are not relevant
dir_to_remove = ['/content/aslamerican-sign-language-aplhabet-dataset/ASL_Alphabet_Dataset/asl_alphabet_train/del',
                 '/content/aslamerican-sign-language-aplhabet-dataset/ASL_Alphabet_Dataset/asl_alphabet_train/nothing',
                 '/content/aslamerican-sign-language-aplhabet-dataset/ASL_Alphabet_Dataset/asl_alphabet_train/space'] # list of unnecessary directories

# Go through the list and remove
for path in dir_to_remove:
    if os.path.exists(path):
        shutil.rmtree(path)

Dataset URL: https://www.kaggle.com/datasets/debashishsau/aslamerican-sign-language-aplhabet-dataset
License(s): CC0-1.0
100% 4.20G/4.20G [03:25<00:00, 24.5MB/s]
100% 4.20G/4.20G [03:25<00:00, 21.9MB/s]


In [4]:
# Set path to the dataset directory
dataset_dir = "/content/aslamerican-sign-language-aplhabet-dataset/ASL_Alphabet_Dataset/asl_alphabet_train"

# Retrieve image paths and corresponding labels from the subdirectory names
image_paths = []
labels = []

# Go through subdirectories
for label in os.listdir(dataset_dir):
    class_dir = os.path.join(dataset_dir, label) # create path to subdirectory
    if os.path.isdir(class_dir): # check if it's a directory, not a file
        print(f"Processing class: {label}") # see processed classes for debugging
        for image_name in os.listdir(class_dir):
            if image_name.endswith((".jpg", ".jpeg", ".png")): # go through only files that are images
                image_path = os.path.join(class_dir, image_name) # create path to file
                image_paths.append(image_path)
                labels.append(label)

Processing class: N
Processing class: R
Processing class: A
Processing class: T
Processing class: D
Processing class: K
Processing class: M
Processing class: Z
Processing class: S
Processing class: W
Processing class: Q
Processing class: V
Processing class: B
Processing class: Y
Processing class: L
Processing class: H
Processing class: I
Processing class: F
Processing class: G
Processing class: U
Processing class: C
Processing class: X
Processing class: O
Processing class: E
Processing class: P
Processing class: J


In [5]:
# Prepare dataset class for preprocessing
class ASLDataset(Dataset):
    def __init__(self, image_paths, labels, processor):
        self.image_paths = image_paths
        self.labels = labels
        self.processor = processor

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        # Load image and corresponding label
        image = Image.open(self.image_paths[idx]).convert("RGB")
        label = self.labels[idx]

        # Preprocess image and text (label) using the CLIPProcessor
        inputs = self.processor(images=image, text=label, return_tensors="pt", padding=True)

        # Return the processed data as a dictionary
        return {
            "pixel_values": inputs["pixel_values"].squeeze(0),
            "input_ids": inputs["input_ids"].squeeze(0),
            "attention_mask": inputs["attention_mask"].squeeze(0)
        }

# Initialize CLIP processor
processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")
dataset = ASLDataset(image_paths, labels, processor)

# # Split the dataset 70/20/10%
# train_size = int(0.7 * len(dataset))
# val_size = int(0.2 * len(dataset))
# test_size = len(dataset) - train_size - val_size
# train_dataset, val_dataset, test_dataset = random_split(dataset, [train_size, val_size, test_size])

# # Check sizes of the split dataset
# print(f"Train size: {train_size}")
# print(f"Validation size: {val_size}")
# print(f"Test size: {test_size}")

# Create Dataloaders for train, validation, and test sets
#batch_size = 32

#train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
#val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
#test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/316 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/592 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/862k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/525k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.22M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/389 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/4.19k [00:00<?, ?B/s]

In [27]:
# Create and split a small subset to test
subset, _ = random_split(dataset, [1000, len(dataset) - 1000]) # random_split to get instances of several letters
sub_val_size = int(0.2 * len(subset))
sub_train_size = len(subset) - sub_val_size
sub_train_dataset, sub_val_dataset = random_split(subset, [sub_train_size, sub_val_size]) #split into 80/20 train and validation sets

# Set batch size and create Dataloaders for the subset
batch_size = 16

sub_train_loader = DataLoader(sub_train_dataset, batch_size=batch_size, shuffle=True)
sub_val_loader = DataLoader(sub_val_dataset, batch_size=batch_size, shuffle=True)

In [7]:
# Check the shape of a batch
batch = next(iter(sub_train_loader))
print(batch)

{'pixel_values': tensor([[[[ 1.7406,  1.7406,  1.7406,  ...,  1.7406,  1.7406,  1.7406],
          [ 1.7406,  1.7406,  1.7406,  ...,  1.7406,  1.7406,  1.7406],
          [ 1.7406,  1.7406,  1.7406,  ...,  1.7406,  1.7406,  1.7406],
          ...,
          [ 1.7406,  1.7406,  1.7406,  ...,  1.7406,  1.7406,  1.7406],
          [ 1.7406,  1.7406,  1.7406,  ...,  1.7406,  1.7406,  1.7406],
          [ 1.7406,  1.7406,  1.7406,  ...,  1.7406,  1.7406,  1.7406]],

         [[ 1.8948,  1.8948,  1.8948,  ...,  1.8948,  1.8948,  1.8948],
          [ 1.8948,  1.8948,  1.8948,  ...,  1.8948,  1.8948,  1.8948],
          [ 1.8948,  1.8948,  1.8948,  ...,  1.8948,  1.8948,  1.8948],
          ...,
          [ 1.8948,  1.8948,  1.8948,  ...,  1.8948,  1.8948,  1.8948],
          [ 1.8948,  1.8948,  1.8948,  ...,  1.8948,  1.8948,  1.8948],
          [ 1.8948,  1.8948,  1.8948,  ...,  1.8948,  1.8948,  1.8948]],

         [[ 2.0037,  2.0037,  2.0037,  ...,  2.0037,  2.0037,  2.0037],
          [ 2

In [8]:
# Initialize the model, criterion, and optimizer
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")

criterion = nn.CrossEntropyLoss()
optimizer = AdamW(model.parameters(), lr=5e-5)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1)

# Device setup
model.to(device)

pytorch_model.bin:   0%|          | 0.00/605M [00:00<?, ?B/s]

CLIPModel(
  (text_model): CLIPTextTransformer(
    (embeddings): CLIPTextEmbeddings(
      (token_embedding): Embedding(49408, 512)
      (position_embedding): Embedding(77, 512)
    )
    (encoder): CLIPEncoder(
      (layers): ModuleList(
        (0-11): 12 x CLIPEncoderLayer(
          (self_attn): CLIPSdpaAttention(
            (k_proj): Linear(in_features=512, out_features=512, bias=True)
            (v_proj): Linear(in_features=512, out_features=512, bias=True)
            (q_proj): Linear(in_features=512, out_features=512, bias=True)
            (out_proj): Linear(in_features=512, out_features=512, bias=True)
          )
          (layer_norm1): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
          (mlp): CLIPMLP(
            (activation_fn): QuickGELUActivation()
            (fc1): Linear(in_features=512, out_features=2048, bias=True)
            (fc2): Linear(in_features=2048, out_features=512, bias=True)
          )
          (layer_norm2): LayerNorm((512,), eps=1e

In [23]:
# Training loop
def train(model, dataloader, criterion, optimizer, scheduler, device):
    model.train()
    running_loss = 0.0
    total_batches = len(dataloader)

    for batch_idx, batch in enumerate(dataloader):
        pixel_values = batch["pixel_values"].to(device)
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)

        optimizer.zero_grad()

        # Forward pass through CLIP
        outputs = model(
            pixel_values=pixel_values,
            input_ids=input_ids,
            attention_mask=attention_mask,
        )

        # Compute logits and loss
        logits_per_image = outputs.logits_per_image
        ground_truth = torch.arange(len(logits_per_image)).to(device)
        loss = criterion(logits_per_image, ground_truth)

        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=5)
        optimizer.step()
        running_loss += loss.item()

    scheduler.step()
    epoch_loss = running_loss / total_batches
    print(f"Average Loss: {epoch_loss:.4f}")

    return epoch_loss

def evaluate(model, dataloader, criterion, device):
    model.eval()  # Set model to evaluation mode
    total_correct = 0
    total_samples = 0
    val_loss = 0.0

    with torch.no_grad():  # No gradients during evaluation
        for batch in dataloader:
            # Move inputs to the device
            pixel_values = batch["pixel_values"].to(device)
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)

            # Forward pass through the model
            outputs = model(
                pixel_values=pixel_values,
                input_ids=input_ids,
                attention_mask=attention_mask
            )

            # Get logits_per_image (image-text similarity scores)
            logits_per_image = outputs.logits_per_image

            # Predicted indices (most similar text for each image)
            predicted_indices = torch.argmax(logits_per_image, dim=1)

            # Ground truth indices
            ground_truth_indices = torch.arange(len(predicted_indices)).to(device)

            # Count correct predictions
            correct_predictions = (predicted_indices == ground_truth_indices).sum().item()
            total_correct += correct_predictions
            total_samples += len(predicted_indices)

            # Compute validation loss
            loss = criterion(logits_per_image, ground_truth_indices)
            val_loss += loss.item()

    # Compute overall metrics
    accuracy = total_correct / total_samples
    avg_val_loss = val_loss / len(dataloader)

    print(f"Validation Loss: {avg_val_loss:.4f}")
    print(f"Validation Accuracy: {accuracy:.4f}")

    return avg_val_loss, accuracy

In [28]:
# Set number of epochs
num_epochs = 10

# Training and evaluation
for epoch in range(num_epochs):
    print(f"Epoch {epoch+1}/{num_epochs}")
    train_loss = train(
        model=model,
        dataloader=tqdm(sub_train_loader, desc=f"Epoch {epoch+1}"),
        criterion=criterion,
        optimizer=optimizer,
        scheduler=scheduler,
        device=device
    )

    val_loss, val_accuracy = evaluate(
    model=model,
    dataloader=sub_val_loader,
    criterion=criterion,
    device=device
    )


Epoch 1/10


Epoch 1: 100%|██████████| 50/50 [00:14<00:00,  3.45it/s]


Average Loss: 1.2718
Validation Loss: 1.1109
Validation Accuracy: 0.6100
Epoch 2/10


Epoch 2: 100%|██████████| 50/50 [00:13<00:00,  3.66it/s]


Average Loss: 1.2367
Validation Loss: 1.1999
Validation Accuracy: 0.6050
Epoch 3/10


...

Average Loss: 1.2716
Validation Loss: 1.1275
Validation Accuracy: 0.6000
Epoch 9/10


Epoch 9: 100%|██████████| 50/50 [00:13<00:00,  3.69it/s]


Average Loss: 1.2418
Validation Loss: 1.1935
Validation Accuracy: 0.5350
Epoch 10/10


Epoch 10: 100%|██████████| 50/50 [00:13<00:00,  3.66it/s]


Average Loss: 1.2718
Validation Loss: 1.1965
Validation Accuracy: 0.5650
